# Headroom Analysis Example

This notebook demonstrates how to perform headroom analysis for health technology assessment.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Define a headroom analysis function
def calculate_headroom(base_cost, base_wtp, new_cost, new_effect, current_price=None):
    """
    Calculate headroom for a new intervention.
    
    Args:
        base_cost: Cost of current standard of care
        base_wtp: Willingness-to-pay threshold
        new_cost: Cost of new intervention
        new_effect: Effectiveness of new intervention
        current_price: Current price of new intervention (if any)
        
    Returns:
        Dictionary with headroom analysis results
    """
    # Calculate base case effectiveness for comparison
    base_effect = base_cost / base_wtp  # Reverse calculation to get base effect
    
    # Calculate ICER
    icer = (new_cost - base_cost) / (new_effect - base_effect)
    
    # Calculate maximum acceptable cost (at WTP threshold)
    max_acceptable_cost = base_cost + base_wtp * (new_effect - base_effect)
    
    # Calculate headroom
    if current_price is not None:
        headroom = max_acceptable_cost - current_price
    else:
        headroom = max_acceptable_cost - new_cost
    
    # Calculate value ratio (benefit / cost)
    value_ratio = new_effect / new_cost
    
    return {
        'icer': icer,
        'max_acceptable_cost': max_acceptable_cost,
        'current_cost': new_cost,
        'headroom': headroom,
        'value_ratio': value_ratio,
        'is_cost_effective': icer <= base_wtp
    }

In [ ]:
# Define example parameters
strategies = ['ECT', 'IV-KA', 'PO-KA']
base_strategy = 'ECT'

# Base case parameters
base_cost = 5000  # Cost of ECT
base_effect = 0.6  # Effectiveness of ECT in QALYs
base_wtp = 50000   # Willingness-to-pay threshold

# New intervention parameters
new_costs = [7500, 6000]  # Costs for IV-KA and PO-KA
new_effects = [0.8, 0.7]  # Effectiveness for IV-KA and PO-KA
current_prices = [7000, 5500]  # Current list prices

# Perform headroom analysis
headroom_results = []

for i, strategy in enumerate(strategies[1:]):  # Skip base strategy
    result = calculate_headroom(
        base_cost, 
        base_wtp, 
        new_costs[i], 
        base_effect + (new_effects[i] - base_effect),  # Effectiveness difference
        current_prices[i]
    )
    
    headroom_results.append({
        'strategy': strategy,
        'base_strategy': base_strategy,
        'icer': result['icer'],
        'max_acceptable_cost': result['max_acceptable_cost'],
        'current_cost': new_costs[i],
        'current_price': current_prices[i],
        'headroom': result['headroom'],
        'value_ratio': result['value_ratio'],
        'is_cost_effective': result['is_cost_effective']
    })

headroom_df = pd.DataFrame(headroom_results)
print(headroom_df)

In [ ]:
# Visualize headroom analysis
fig, ax = plt.subplots(2, 2, figsize=(15, 12))

# Plot 1: Max acceptable cost vs current cost
x = np.arange(len(headroom_df))
width = 0.35

ax[0, 0].bar(x - width/2, headroom_df['max_acceptable_cost'], width, label='Max Acceptable Cost', alpha=0.7)
ax[0, 0].bar(x + width/2, headroom_df['current_price'], width, label='Current Price', alpha=0.7)
ax[0, 0].set_xlabel('Strategy')
ax[0, 0].set_ylabel('Cost ($AUD)')
ax[0, 0].set_title('Max Acceptable Cost vs Current Price')
ax[0, 0].set_xticks(x)
ax[0, 0].set_xticklabels(headroom_df['strategy'])
ax[0, 0].legend()
ax[0, 0].grid(True, alpha=0.3)

# Add headroom annotations
for i, (_, row) in enumerate(headroom_df.iterrows()):
    headroom = row['headroom']
    if headroom > 0:
        ax[0, 0].text(i, row['max_acceptable_cost'] + 500, f'{headroom:.0f}', ha='center', va='bottom')
    else:
        ax[0, 0].text(i, row['current_price'] + 500, f'{headroom:.0f}', ha='center', va='bottom')

# Plot 2: Headroom as a bar chart
colors = ['green' if h > 0 else 'red' for h in headroom_df['headroom']]
ax[0, 1].bar(headroom_df['strategy'], headroom_df['headroom'], color=colors, alpha=0.7)
ax[0, 1].axhline(y=0, color='black', linestyle='--', alpha=0.7)
ax[0, 1].set_xlabel('Strategy')
ax[0, 1].set_ylabel('Headroom ($AUD)')
ax[0, 1].set_title('Price Headroom by Strategy')
ax[0, 1].grid(True, alpha=0.3)

# Plot 3: ICER vs WTP threshold
ax[1, 0].bar(headroom_df['strategy'], headroom_df['icer'], alpha=0.7, label='ICER')
ax[1, 0].axhline(y=base_wtp, color='red', linestyle='--', label=f'WTP Threshold (${base_wtp:,})', alpha=0.7)
ax[1, 0].set_xlabel('Strategy')
ax[1, 0].set_ylabel('ICER ($/QALY)')
ax[1, 0].set_title('ICER vs Willingness-to-Pay Threshold')
ax[1, 0].legend()
ax[1, 0].grid(True, alpha=0.3)

# Add cost-effectiveness labels
for i, (_, row) in enumerate(headroom_df.iterrows()):
    ce_status = 'Cost-Effective' if row['is_cost_effective'] else 'Not Cost-Effective'
    ax[1, 0].text(i, row['icer'] + 2000, ce_status, ha='center', va='bottom', fontsize=9)

# Plot 4: Value ratio (effectiveness/cost)
ax[1, 1].bar(headroom_df['strategy'], headroom_df['value_ratio'], color=['green', 'orange'], alpha=0.7)
ax[1, 1].set_xlabel('Strategy')
ax[1, 1].set_ylabel('Value Ratio (QALYs/$)')
ax[1, 1].set_title('Value Ratio by Strategy')
ax[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Sensitivity analysis on headroom with different WTP thresholds
wtp_range = np.arange(30000, 80001, 5000)
sensitivity_data = []

for wtp in wtp_range:
    for i, strategy in enumerate(headroom_df['strategy']):
        result = calculate_headroom(
            base_cost, 
            wtp, 
            new_costs[i], 
            base_effect + (new_effects[i] - base_effect),
            current_prices[i]
        )
        
        sensitivity_data.append({
            'strategy': strategy,
            'wtp': wtp,
            'icer': result['icer'],
            'max_acceptable_cost': result['max_acceptable_cost'],
            'headroom': result['headroom'],
            'is_cost_effective': result['is_cost_effective']
        })

sensitivity_df = pd.DataFrame(sensitivity_data)

# Plot sensitivity analysis
plt.figure(figsize=(14, 10))

# Headroom sensitivity
plt.subplot(2, 2, 1)
for strategy in headroom_df['strategy']:
    strategy_data = sensitivity_df[sensitivity_df['strategy'] == strategy]
    plt.plot(strategy_data['wtp'], strategy_data['headroom'], label=strategy, linewidth=2)
plt.axhline(y=0, color='red', linestyle='--', label='Break-even', alpha=0.7)
plt.xlabel('WTP Threshold ($/QALY)')
plt.ylabel('Headroom ($AUD)')
plt.title('Headroom Sensitivity to WTP Threshold')
plt.legend()
plt.grid(True, alpha=0.3)

# ICER sensitivity
plt.subplot(2, 2, 2)
for strategy in headroom_df['strategy']:
    strategy_data = sensitivity_df[sensitivity_df['strategy'] == strategy]
    plt.plot(strategy_data['wtp'], strategy_data['icer'], label=strategy, linewidth=2)
wtp_values = sensitivity_df['wtp'].unique()
plt.plot(wtp_values, wtp_values, 'k--', label='Line of Equality', alpha=0.7)
plt.xlabel('WTP Threshold ($/QALY)')
plt.ylabel('ICER ($/QALY)')
plt.title('ICER Sensitivity to WTP Threshold')
plt.legend()
plt.grid(True, alpha=0.3)

# Max acceptable cost sensitivity
plt.subplot(2, 2, 3)
for strategy in headroom_df['strategy']:
    strategy_data = sensitivity_df[sensitivity_df['strategy'] == strategy]
    plt.plot(strategy_data['wtp'], strategy_data['max_acceptable_cost'], label=strategy, linewidth=2)
plt.axhline(y=current_prices[0], color='blue', linestyle=':', label='IV-KA Current Price', alpha=0.7)
plt.axhline(y=current_prices[1], color='orange', linestyle=':', label='PO-KA Current Price', alpha=0.7)
plt.xlabel('WTP Threshold ($/QALY)')
plt.ylabel('Max Acceptable Cost ($AUD)')
plt.title('Max Acceptable Cost Sensitivity to WTP Threshold')
plt.legend()
plt.grid(True, alpha=0.3)

# Cost-effectiveness acceptability
plt.subplot(2, 2, 4)
for strategy in headroom_df['strategy']:
    strategy_data = sensitivity_df[sensitivity_df['strategy'] == strategy]
    ce_proportion = strategy_data.groupby('wtp')['is_cost_effective'].mean()
    plt.plot(ce_proportion.index, ce_proportion.values, label=strategy, linewidth=2)
plt.xlabel('WTP Threshold ($/QALY)')
plt.ylabel('Probability Cost-Effective')
plt.title('Cost-Effectiveness Acceptability')
plt.legend()
plt.grid(True, alpha=0.3)
plt.ylim(0, 1)

plt.tight_layout()
plt.show()

In [ ]:
# Create a headroom analysis report
print("="*70)
print("HEADROOM ANALYSIS REPORT")
print("="*70)
print(f"Base Strategy: {base_strategy}")
print(f"Base Cost: ${base_cost:,}")
print(f"Base Effectiveness: {base_effect} QALYs")
print(f"WTP Threshold: ${base_wtp:,}/QALY")
print()

for _, row in headroom_df.iterrows():
    print(f"Strategy: {row['strategy']}")
    print(f"  Current Price: ${row['current_price']:,}")
    print(f"  Max Acceptable Cost: ${row['max_acceptable_cost']:,.0f}")
    print(f"  Headroom: ${row['headroom']:,.0f}")
    print(f"  ICER: ${row['icer']:,.0f}/QALY")
    print(f"  Cost-Effective: {'Yes' if row['is_cost_effective'] else 'No'}")
    
    if row['headroom'] > 0:
        print(f"  \033[92m  Recommendation: Price can increase by ${row['headroom']:,.0f} and remain cost-effective\033[0m")
    else:
        print(f"  \033[91m  Recommendation: Price needs to decrease by ${abs(row['headroom']):,.0f} to be cost-effective\033[0m")
    print()

## Next Steps

1. Integrate with actual cost and outcome data
2. Perform probabilistic sensitivity analysis
3. Consider budget impact constraints
4. Evaluate headroom under different market scenarios